In [1]:
!pip install transformers[sentencepiece]
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [2]:
# Import necessary libraries

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
import evaluate
import numpy as np
import torch

2026-02-10 12:49:12.434099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770727752.625491      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770727752.689227      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770727753.179218      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770727753.179279      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770727753.179282      55 computation_placer.cc:177] computation placer alr

In [3]:
# Define Model and the tokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Data Preprocessing and Cleaning

spam_dataset = load_dataset("sms_spam")
print(spam_dataset)
print(spam_dataset['train'].features)

train_val = spam_dataset['train'].train_test_split(test_size=0.3, seed=42)
val_test = train_val['test'].train_test_split(test_size=0.5, seed=42)

# As the spam_dataset doesnt have an eval and test dataset, we need to create them.

train_dataset = train_val['train']
val_dataset = val_test['train']
test_dataset = val_test['test']

print(f"Train: {len(train_dataset)}")
print(f"Val: {len(val_dataset)}")
print(f"Test: {len(test_dataset)}")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5574
    })
})
{'sms': Value('string'), 'label': ClassLabel(names=['ham', 'spam'])}
Train: 3901
Val: 836
Test: 837


In [5]:
# Define preprocessing function

def preprocessing_function(example):
    return tokenizer(example['sms'],truncation = True)

In [6]:
# Map the function to all datasets

tokenized_dataset_train = train_dataset.map(preprocessing_function, batched = True)
tokenized_dataset_test = test_dataset.map(preprocessing_function, batched = True)
tokenized_dataset_eval = val_dataset.map(preprocessing_function, batched = True)

Map:   0%|          | 0/3901 [00:00<?, ? examples/s]

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

Map:   0%|          | 0/836 [00:00<?, ? examples/s]

In [7]:
# Define DataCollator for dynamic padding

print(tokenized_dataset_train)

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

Dataset({
    features: ['sms', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3901
})


In [8]:
# Define evaluation metrics to be passed on to the trainer

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"]
    }

In [9]:
# Training arguments for custom training

training_args = TrainingArguments(
    output_dir = "./results",
    eval_strategy = 'epoch',
    fp16 = True,
    learning_rate = 2e-5,
    lr_scheduler_type = "cosine",
    logging_strategy="epoch",
    num_train_epochs = 3,
    weight_decay = 0.01,
    report_to=[]
)

In [11]:
# Check whether training is on CPU or GPU

import torch
torch.cuda.is_available()

True

In [12]:
# Train the model on our data and get evaluation metrics

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_dataset_train,
    eval_dataset = tokenized_dataset_eval,
    data_collator = data_collator,
    processing_class = tokenizer,
    compute_metrics = compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.074600,0.023273,0.996411,0.986784
2,0.018800,0.038624,0.992823,0.973451
3,0.005100,0.040971,0.994019,0.977974


TrainOutput(global_step=1464, training_loss=0.03284003907214097, metrics={'train_runtime': 107.3653, 'train_samples_per_second': 109.002, 'train_steps_per_second': 13.636, 'total_flos': 328135458248520.0, 'train_loss': 0.03284003907214097, 'epoch': 3.0})

In [13]:
# Initial Testing

texts = [
    "Free entry in a weekly contest. Click here to learn more!",
    "Hey, are we meeting for lunch today?"
]

inputs = tokenizer(texts, truncation = True, padding = True, return_tensors = "pt").to(trainer.model.device)

with torch.no_grad():
    outputs = trainer.model(**inputs)
    logits = outputs.logits

predictions = torch.argmax(logits,dim=-1)
print(predictions)

label_map = {0:"ham",1:"spam"}
pred_labels = [label_map[p.item()] for p in predictions]
print(pred_labels)

tensor([1, 0], device='cuda:0')
['spam', 'ham']


In [16]:
# Testing

texts = test_dataset['sms']
all_texts = list(all_texts)
inputs = tokenizer(all_texts, truncation = True, padding = True, return_tensors = "pt").to(trainer.model.device)

metrics = trainer.evaluate(eval_dataset = tokenized_dataset_eval)
print(metrics)

{'eval_loss': 0.040971435606479645, 'eval_accuracy': 0.9940191387559809, 'eval_f1': 0.9779735682819384, 'eval_runtime': 1.9888, 'eval_samples_per_second': 420.345, 'eval_steps_per_second': 52.795, 'epoch': 3.0}
